In [1]:
import pandas as pd
import json
import regex as re

In [2]:
df = pd.read_csv('../dataset_opengen.csv')

In [3]:
def remove_script_tags(text):
    # Pattern to match <script> tags and their content
    pattern = r'<script\b[^<]*(?:(?!<\/script>)<[^<]*)*<\/script>'
    # Replace the matched patterns with an empty string
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return cleaned_text

def parse_for_dataset(user_text, assistant_text):
    # sys_prompt="<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a Front End assistant, you will be prompted a description of web component/website and you should only return the code in HTML and TailwindCSS. DO NOT USE  HEADER TAGS OR ANY SCRIPT TAGS JUST THE HTML AND TAILWINDCSS CODE<|eot_id|>"
    sys_prompt="<s>"
    u_start="<|user|>\n"
    u_end="<|end|>\n"
    a_start="<|assistant|>\n```html"
    a_end = "```<|end|>\n"

    result = sys_prompt+u_start+user_text+u_end + \
        a_start+remove_script_tags(assistant_text)+a_end
    return {"text": result}

In [4]:
dataset={}
import json
file_path = 'dataset_Phi-3.json'
for index, row in df.iterrows():
    u_prompt = row['u_desc']
    code = row['code']
    with open(file_path, 'a') as file:
        
        # Convert to JSON and append to the file
        json_string = json.dumps(parse_for_dataset(u_prompt, code))
        file.write(json_string + '\n')  # Ensure newline for JSONL format
    # Now you can use u_value and t_value as needed

In [5]:
import json


def split_jsonl_file(original_file, first_file, second_file):
    # Initialize counters
    count = 0

    # Open the first and second files for writing
    with open(first_file, 'w') as file1, open(second_file, 'w') as file2:
        # Step 1: Read the original .jsonl file
        with open(original_file, 'r') as file:
            for line in file:
                # Parse the JSON object from the line
                data = json.loads(line)

                # Step 2: Split the data and Step 3: Write to the appropriate .jsonl file
                if count < 200:
                    # Write to the first file
                    json.dump(data, file1)
                    file1.write('\n')
                else:
                    # Write to the second file
                    json.dump(data, file2)
                    file2.write('\n')

                # Increment the counter
                count += 1


# Example usage
split_jsonl_file('dataset_Phi-3.json', 'valid.json', 'train.json')